# 🔍 Reformer: Efficient via LSH

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gaurav-redhat/transformer_problems/blob/main/transformer_architectures/08_reformer/demo.ipynb)

![Architecture](architecture.png)

### Key Innovations
- **LSH Attention**: Locality Sensitive Hashing groups similar queries
- **Reversible Layers**: Recompute activations, no memory storage
- **Chunked FFN**: Process in chunks to save memory

In [ ]:
!pip install torch matplotlib numpy -q
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

## Locality Sensitive Hashing (LSH)

Core idea: **Similar vectors should hash to the same bucket**

For attention: Only compute attention within each bucket!

In [ ]:
def angular_lsh(x, n_hashes, n_buckets):
    """Angular LSH: hash vectors by their direction."""
    d = x.shape[-1]
    
    # Random projection vectors
    projections = torch.randn(n_hashes, d, n_buckets // 2, device=x.device)
    projections = projections / projections.norm(dim=1, keepdim=True)
    
    # Project and sign -> bucket assignment
    # x: (B, N, d) @ projections: (n_hashes, d, n_buckets//2) -> (n_hashes, B, N, n_buckets//2)
    dots = torch.einsum('bnd,hdk->hbnk', x, projections)
    
    # Convert to bucket index: sign determines which half of buckets
    signs = (dots > 0).int()
    buckets = signs.sum(dim=-1)  # Simple hash combining
    
    return buckets  # (n_hashes, B, N)

# Visualize LSH
def visualize_lsh():
    # Create 2D points
    n_points = 100
    points = torch.randn(1, n_points, 2)
    
    # Compute LSH buckets
    buckets = angular_lsh(points, n_hashes=1, n_buckets=8)[0, 0]  # (N,)
    
    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Original points
    ax = axes[0]
    ax.scatter(points[0, :, 0], points[0, :, 1], c='gray', alpha=0.6)
    ax.set_title('Original Points')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    
    # Points colored by bucket
    ax = axes[1]
    scatter = ax.scatter(points[0, :, 0], points[0, :, 1], c=buckets, cmap='tab10', alpha=0.8)
    ax.set_title('Points Colored by LSH Bucket')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    plt.colorbar(scatter, ax=ax, label='Bucket')
    
    plt.tight_layout()
    plt.show()
    
    print('Similar vectors (close in direction) tend to be in the same bucket!')

visualize_lsh()

## LSH Attention

In [ ]:
class LSHAttention(nn.Module):
    """Locality Sensitive Hashing Attention."""
    def __init__(self, d_model, n_heads, n_buckets=64, n_hashes=4, dropout=0.1):
        super().__init__()
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        self.n_buckets = n_buckets
        self.n_hashes = n_hashes
        self.scale = self.d_k ** -0.5
        
        # Shared Q and K (Reformer uses Q=K for LSH efficiency)
        self.W_qk = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def _lsh_hash(self, x, n_buckets):
        """Simple LSH hash function."""
        d = x.shape[-1]
        # Random projections (in practice, should be fixed per layer)
        projections = torch.randn(d, n_buckets // 2, device=x.device)
        projections = projections / projections.norm(dim=0, keepdim=True)
        
        dots = x @ projections  # (B, H, N, n_buckets//2)
        buckets = (dots > 0).int()
        # Convert to bucket indices
        powers = 2 ** torch.arange(n_buckets // 2, device=x.device)
        bucket_ids = (buckets * powers).sum(dim=-1) % n_buckets
        return bucket_ids
    
    def forward(self, x):
        B, T, C = x.shape
        
        # Project to QK (shared) and V
        qk = self.W_qk(x).view(B, T, self.n_heads, self.d_k).transpose(1, 2)
        V = self.W_v(x).view(B, T, self.n_heads, self.d_k).transpose(1, 2)
        
        # Normalize QK for LSH
        qk_normalized = F.normalize(qk, dim=-1)
        
        # Get bucket assignments
        buckets = self._lsh_hash(qk_normalized, self.n_buckets)  # (B, H, T)
        
        # For simplicity, we'll do full attention here but in practice:
        # Only attend within same bucket + allow attention to adjacent buckets
        
        # Full attention (simplified for demonstration)
        attn = (qk @ qk.transpose(-2, -1)) * self.scale
        
        # Mask: only attend to tokens in same/similar buckets
        bucket_mask = (buckets.unsqueeze(-1) == buckets.unsqueeze(-2))  # (B, H, T, T)
        # Allow some cross-bucket attention by relaxing mask
        attn = attn.masked_fill(~bucket_mask, -1e4)  # Soft mask
        
        # Causal mask
        causal_mask = torch.tril(torch.ones(T, T, device=x.device)).bool()
        attn = attn.masked_fill(~causal_mask, float('-inf'))
        
        attn = F.softmax(attn, dim=-1)
        attn = self.dropout(attn)
        
        out = (attn @ V).transpose(1, 2).reshape(B, T, C)
        return self.W_o(out), buckets

# Test LSH attention
lsh_attn = LSHAttention(d_model=64, n_heads=4, n_buckets=8)
x = torch.randn(2, 32, 64)
out, buckets = lsh_attn(x)
print(f'Input: {x.shape}')
print(f'Output: {out.shape}')
print(f'Bucket assignments: {buckets.shape}')
print(f'Unique buckets used: {buckets[0, 0].unique().shape[0]}')

## Reversible Layers

Key memory saving: Don't store intermediate activations!

In [ ]:
class ReversibleBlock(nn.Module):
    """Reversible residual block - can recompute inputs from outputs."""
    def __init__(self, f, g):
        super().__init__()
        self.f = f  # Attention
        self.g = g  # FFN
    
    def forward(self, x1, x2):
        """Forward: y1 = x1 + f(x2), y2 = x2 + g(y1)"""
        y1 = x1 + self.f(x2)
        y2 = x2 + self.g(y1)
        return y1, y2
    
    def backward_pass(self, y1, y2):
        """Reverse: recover x1, x2 from y1, y2"""
        x2 = y2 - self.g(y1)
        x1 = y1 - self.f(x2)
        return x1, x2

# Demonstrate reversibility
def demonstrate_reversibility():
    d_model = 64
    
    # Simple functions for demonstration
    f = nn.Sequential(nn.LayerNorm(d_model), nn.Linear(d_model, d_model), nn.GELU())
    g = nn.Sequential(nn.LayerNorm(d_model), nn.Linear(d_model, d_model), nn.GELU())
    
    block = ReversibleBlock(f, g)
    
    # Input
    x1 = torch.randn(2, 16, d_model)
    x2 = torch.randn(2, 16, d_model)
    
    print('Original x1[:, 0, :5]:', x1[0, 0, :5].tolist())
    
    # Forward
    with torch.no_grad():
        y1, y2 = block(x1, x2)
        
        # Backward (reconstruct)
        x1_reconstructed, x2_reconstructed = block.backward_pass(y1, y2)
    
    print('Reconstructed x1[:, 0, :5]:', x1_reconstructed[0, 0, :5].tolist())
    
    error = (x1 - x1_reconstructed).abs().max().item()
    print(f'\nReconstruction error: {error:.2e}')
    print('Perfect reconstruction means no need to store activations!')

demonstrate_reversibility()

## Reformer Implementation

In [ ]:
class ReformerBlock(nn.Module):
    def __init__(self, d_model, n_heads, n_buckets=32, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(d_model)
        self.attn = LSHAttention(d_model, n_heads, n_buckets, dropout=dropout)
        self.norm2 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.GELU(),
            nn.Linear(4 * d_model, d_model),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        attn_out, _ = self.attn(self.norm1(x))
        x = x + attn_out
        x = x + self.ff(self.norm2(x))
        return x

class Reformer(nn.Module):
    def __init__(self, vocab_size, d_model=128, n_heads=4, n_layers=3, n_buckets=32, max_len=512, dropout=0.1):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = nn.Embedding(max_len, d_model)
        self.dropout = nn.Dropout(dropout)
        
        self.layers = nn.ModuleList([ReformerBlock(d_model, n_heads, n_buckets, dropout) for _ in range(n_layers)])
        self.norm = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size)
    
    def forward(self, x):
        B, T = x.shape
        pos = torch.arange(T, device=x.device).unsqueeze(0)
        
        x = self.dropout(self.embed(x) + self.pos_embed(pos))
        for layer in self.layers:
            x = layer(x)
        
        return self.head(self.norm(x))

model = Reformer(vocab_size=1000, d_model=64, n_heads=4, n_layers=2, n_buckets=16).to(device)
print(f'Reformer Parameters: {sum(p.numel() for p in model.parameters()):,}')

## Training Reformer

In [ ]:
# Dataset
text = 'the quick brown fox jumps over the lazy dog ' * 300
chars = sorted(list(set(text)))
vocab_size = len(chars)
char_to_idx = {c: i for i, c in enumerate(chars)}
data = torch.tensor([char_to_idx[c] for c in text], dtype=torch.long)

# Training
seq_len = 64
model = Reformer(vocab_size=vocab_size, d_model=64, n_heads=4, n_layers=2, n_buckets=16, max_len=seq_len).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

losses = []
n_steps = 300

print('Training Reformer with LSH attention...')
for step in range(n_steps):
    idx = torch.randint(0, len(data) - seq_len - 1, (16,))
    x = torch.stack([data[i:i+seq_len] for i in idx]).to(device)
    y = torch.stack([data[i+1:i+seq_len+1] for i in idx]).to(device)
    
    optimizer.zero_grad()
    logits = model(x)
    loss = F.cross_entropy(logits.view(-1, vocab_size), y.view(-1))
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    if (step + 1) % 50 == 0:
        print(f'Step {step+1}: Loss = {loss.item():.4f}')

plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Step')
plt.ylabel('Loss')
plt.title('Reformer Training (LSH Attention)')
plt.grid(True, alpha=0.3)
plt.show()

print('\n🎯 Key Takeaways:')
print('1. LSH groups similar queries into buckets')
print('2. Only compute attention within buckets → O(N log N)')
print('3. Reversible layers eliminate activation storage')
print('4. Memory-efficient for very long sequences')